## Experiments: Sensitive Analysis Hyperparameter RSAST:


It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 30 ,50 and 100 for the selected dataset.

In [2]:
#configure directory to import sast libraries
import sys 
import os 
#add sast library path
file_path = os.path.expanduser("~/random_sast/sast")
sys.path.append(file_path)
#add cd_diagram library path
file_path = os.path.expanduser("~/random_sast/cd_diagram")
sys.path.append(file_path)

#sys.path

In [3]:
from sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd
import os

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/mass_ts/_mass_ts.py:17: UserWarning: GPU support will not work. You must pip install mass-ts[gpu].
  warnings.warn(


### Reading Datasets

In [4]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsByClassifier'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)
df_other_methods=df_other_methods[['filename','folds:','0']]
df_other_methods['method']=df_other_methods['filename'].str.split('_').str[0]
df_other_methods
df_other_methods=df_other_methods.rename(columns={'folds:':'dataset','0':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]


In [5]:
df_other_methods['method'].unique()

array(['SAST', 'ROCKET', 'HIVE-COTEv1', 'BOSS', 'RISE', 'TSF',
       'InceptionTime', 'TS-CHIEF', 'S-BOSS', 'STC', 'ResNet',
       'ProximityForest', 'WEASEL', 'Catch22', 'cBOSS'], dtype=object)

In [6]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsRsast'
#directory = os.getcwd()+'/results_accuracy_per_ds'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.startswith("df_overall_tunning") and filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        df['rpoint']=df['rpoint'].astype(str)
        df['nb_per_class']=df['nb_per_class'].astype(str)

        
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)
df_result.head(3)


,Unnamed: 0,accuracy,time,cweights_time,fsubsequence_time,tdataset_time,tclassifier_time,dataset_name,classifier_name,rpoint,nb_per_class,method,len_method
0,0,0.919203,0.181923,0.013529,0.127419,0.037550,0.002808,SonyAIBORobotSurface2,ACF&PACF: n_random_points=1 nb_inst_per_class=...,1,(max instances per class)//2,Rsast,ACF&PACF
1,1,0.828961,0.168610,0.021266,0.037572,0.106363,0.002882,SonyAIBORobotSurface2,ACF&PACF: n_random_points= (lenthg ts)//2 nb_i...,(lenthg ts)//2,1,Rsast,ACF&PACF
2,2,0.876180,1.377738,0.011656,0.138769,1.211099,0.015296,SonyAIBORobotSurface2,ACF&PACF: n_random_points= (lenthg ts)//2 nb_i...,(lenthg ts)//2,(max instances per class)//2,Rsast,ACF&PACF


In [7]:
#df_result=pd.read_csv("df_overall_tunning_SmoothSubspace3.csv")

df_result['dataset_name'].unique()


array(['SonyAIBORobotSurface2', 'GunPoint', 'Chinatown',
       'DistalPhalanxOutlineCorrect', 'SyntheticControl',
       'SmoothSubspace', 'Coffee', 'DistalPhalanxOutlineAgeGroup',
       'ShapeletSim', 'ItalyPowerDemand'], dtype=object)

In [8]:
df_result.columns.unique()

Index(['Unnamed: 0', 'accuracy', 'time', 'cweights_time', 'fsubsequence_time',
       'tdataset_time', 'tclassifier_time', 'dataset_name', 'classifier_name',
       'rpoint', 'nb_per_class', 'method', 'len_method'],
      dtype='object')

### Hyperparameter Tunning RSAST 

#### Accuracy: Subsequence Lenght Method 

In [9]:

filter_param=df_result[df_result.rpoint=='100']
filter_param.rpoint.unique()

array(['100'], dtype=object)

In [10]:

filter_param=filter_param[filter_param.nb_per_class=='100']
filter_param.nb_per_class.unique()

array(['100'], dtype=object)

In [11]:
# create a pivot table with the mean of score by dataset

len_method_vs_ds=pd.pivot_table(filter_param, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='mean')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
df_rocket=df_other_methods[df_other_methods["method"]=="ROCKET"]
merged_df = len_method_vs_ds.merge(df_rocket,left_on='dataset_name', right_on='dataset',  how='left')
merged_df=merged_df.drop('dataset',axis=1)
merged_df

,dataset_name,ACF,ACF&PACF,Max ACF,Max PACF,None,PACF,score,method
0,Chinatown,0.972789,0.982507,0.967930,0.985423,0.967930,0.982507,0.982507,ROCKET
1,DistalPhalanxOutlineAgeGroup,0.750600,0.721823,0.755396,0.714628,0.705036,0.690647,0.748201,ROCKET
2,DistalPhalanxOutlineCorrect,0.657005,0.723430,0.665459,0.699275,0.741546,0.713768,0.771739,ROCKET
3,GunPoint,0.815556,0.980000,0.857778,0.973333,0.973333,0.977778,1.000000,ROCKET
4,ItalyPowerDemand,0.954001,0.948170,0.955620,0.945254,0.952057,0.949466,0.968902,ROCKET
5,ShapeletSim,0.985185,0.883333,0.985185,0.820370,0.551852,0.883333,1.000000,ROCKET
6,SmoothSubspace,0.937778,0.922222,0.922222,0.926667,0.917778,0.942222,0.980000,ROCKET
7,SonyAIBORobotSurface2,0.855894,0.915005,0.856593,0.879328,0.859741,0.913956,0.916055,ROCKET
8,SyntheticControl,0.937778,0.970000,0.932222,0.967778,0.988889,0.972222,1.000000,ROCKET


In [12]:
# create a pivot table with the mean of score by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.876124,0.767598,0.830243,0.850711,0.849897,0.846883
ACF&PACF,0.941437,0.863064,0.887333,0.892342,0.890511,0.891722
Max ACF,0.883156,0.766028,0.829734,0.856769,0.850126,0.858234
Max PACF,0.928271,0.822022,0.872097,0.872176,0.875258,0.872250
None,0.918005,0.813788,0.847095,0.858963,0.845799,0.860784
PACF,0.941443,0.868973,0.886449,0.891958,0.894048,0.892124


In [13]:
# create a pivot table with the mean of score by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.871440,0.735009,0.835530,0.862523,0.858978,0.858256
ACF&PACF,0.943022,0.846118,0.884652,0.896929,0.897701,0.897892
Max ACF,0.876747,0.742684,0.837514,0.863254,0.862141,0.862089
Max PACF,0.928934,0.808656,0.872263,0.872412,0.882951,0.876818
None,0.920684,0.785547,0.850379,0.860412,0.866524,0.860728
PACF,0.942424,0.859806,0.886081,0.894813,0.895127,0.896685


#### Time: Subsequence Lenght Method 

In [14]:
# create a pivot table with the mean of the time by dataset
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['dataset_name'], aggfunc='mean')

dataset_name,Chinatown,Coffee,DistalPhalanxOutlineAgeGroup,DistalPhalanxOutlineCorrect,GunPoint,ItalyPowerDemand,ShapeletSim,SmoothSubspace,SonyAIBORobotSurface2,SyntheticControl
len_method,,,,,,,,,,
ACF,0.073226,63.990256,111.545551,144.720377,37.041264,0.353901,7.361113,0.561044,0.700003,90.876566
ACF&PACF,0.156653,194.147516,294.637278,356.864070,71.945396,0.729001,146.904885,0.930945,2.032898,211.081948
Max ACF,0.076082,4.597660,40.968521,47.509385,4.165960,0.361998,2.827787,0.565765,0.548111,32.034245
Max PACF,0.069733,7.392416,19.410193,21.999060,4.032023,0.387409,66.417229,0.639174,0.413045,16.692008
None,0.054860,2.293117,18.884042,21.533008,2.376824,0.331860,1.788605,0.523768,0.305284,15.885983
PACF,0.106152,159.124152,202.722461,237.791324,47.107583,0.711325,140.841731,0.843548,1.655537,160.264361


In [15]:
# create a pivot table with the mean of the time by number of instances per class
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,8.549080,2.105433,14.788724,88.785992,39.786102,58.688080
ACF&PACF,20.871311,5.584999,51.131631,233.355543,111.035297,157.699177
Max ACF,0.970739,1.139325,4.773058,28.724759,12.166905,18.429675
Max PACF,1.309154,1.736374,11.107771,21.535387,14.328103,16.890209
None,0.633373,0.878233,2.509438,13.500885,5.857025,8.519595
PACF,15.163705,4.418280,40.069347,167.301683,81.746226,114.630409


In [16]:
# create a pivot table with the mean of the time by number of random points
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,14.499147,1.917038,12.204271,88.277589,35.823054,59.627673
ACF&PACF,35.846036,11.403014,42.040682,212.538634,109.313830,167.643230
Max ACF,1.494305,0.968907,4.453129,27.709595,11.926564,19.620754
Max PACF,1.824314,7.794245,9.697883,16.891515,14.196344,16.471990
None,0.921500,0.791515,2.663375,11.605545,6.487448,9.411992
PACF,26.565756,10.197319,31.942580,147.230880,82.882909,123.830615


#### Variance: Subsequence Lenght Method 

In [17]:
# create a pivot table with the variance of the accuracy by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.008351,0.021427,0.012327,0.012127,0.011181,0.011517
ACF&PACF,0.001372,0.009988,0.008776,0.009016,0.008739,0.009001
Max ACF,0.008521,0.019946,0.012617,0.011987,0.010253,0.011863
Max PACF,0.001919,0.020445,0.009279,0.011375,0.009472,0.010182
None,0.003460,0.019609,0.016377,0.016908,0.016783,0.017675
PACF,0.001433,0.009306,0.008641,0.009816,0.009120,0.009544


In [18]:
# create a pivot table with the variance of the accuracy by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='var')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.007873,0.020787,0.014318,0.013139,0.013436,0.013759
ACF&PACF,0.002009,0.010459,0.008743,0.009000,0.009273,0.008938
Max ACF,0.007557,0.022352,0.014370,0.012111,0.011617,0.011426
Max PACF,0.002291,0.015573,0.011618,0.011991,0.010497,0.012417
None,0.004282,0.019926,0.018493,0.016293,0.018874,0.016429
PACF,0.001839,0.010500,0.009125,0.009167,0.008605,0.009253


#### Another Results

In [19]:
df_result[['dataset_name','len_method','classifier_name']].describe()

,dataset_name,len_method,classifier_name
count,5490,5490,5490
unique,10,6,216
top,SonyAIBORobotSurface2,ACF&PACF,Max ACF: n_random_points= (lenthg ts)//2 nb_in...
freq,720,915,80


In [20]:
pd.pivot_table(df_result, values='time', index=['dataset_name'], aggfunc='sum')/(60*60)

,time
dataset_name,
Chinatown,0.017890
Coffee,1.798105
DistalPhalanxOutlineAgeGroup,14.336834
DistalPhalanxOutlineCorrect,17.300359
GunPoint,5.555635
ItalyPowerDemand,0.095850
ShapeletSim,7.627945
SmoothSubspace,0.135475
SonyAIBORobotSurface2,0.188496


In [21]:
df_result['time'].sum()/(60*60)

58.03231984555721

#### Generate CD Diagram Tunning Hyperparameter

In [22]:
#create dataset to generate cd diagram
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
#df_perf.classifier_name.unique()


In [23]:
param_per_ds=pd.pivot_table(df_perf, index=['dataset_name'],values=['classifier_name'] ,aggfunc='count')
param_per_ds=param_per_ds.reset_index()
complete_ds=param_per_ds[param_per_ds.classifier_name==max(param_per_ds.classifier_name)]["dataset_name"].unique()

df_perf=df_perf[df_perf.dataset_name.isin(complete_ds)]
df_result=df_result[df_result.dataset_name.isin(complete_ds)]
df_perf.dataset_name.unique()

array(['Chinatown', 'GunPoint', 'ItalyPowerDemand', 'SmoothSubspace',
       'SonyAIBORobotSurface2'], dtype=object)

In [24]:
from cd_function import *
#generate hyperparameter tuning cd diagrams
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF&PACF:')], labels=True, title="ACF&PACF comparison", name_file='images_cd_diagram/cd-diagram_ACF&PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^PACF:')], labels=True, title="PACF comparison", name_file='images_cd_diagram/cd-diagram_PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF:')], labels=True, title="ACF comparison", name_file='images_cd_diagram/cd-diagram_ACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^None:')], labels=True, title="None comparison", name_file='images_cd_diagram/cd-diagram_None.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max ACF:')], labels=True, title="Max ACF comparison", name_file='images_cd_diagram/cd-diagram_MaxACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max PACF:')], labels=True, title="Max PACF comparison", name_file='images_cd_diagram/cd-diagram_MaxPACF.png')
draw_cd_diagram(df_perf, labels=True, title="overall Hyperparameter Tunning", name_file='images_cd_diagram/cd-diagram_overall_tunning.png')



/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small 

In [25]:
_, average_ranks, _ = wilcoxon_holm(df_perf=df_perf)
min_rank= min(average_ranks)
average_ranks=pd.DataFrame(average_ranks)
average_ranks

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


,0
ACF: n_random_points=1 nb_inst_per_class=1,214.6
Max ACF: n_random_points=1 nb_inst_per_class=1,214.4
Max PACF: n_random_points=1 nb_inst_per_class=1,209.3
ACF: n_random_points=10 nb_inst_per_class=1,206.3
None: n_random_points=1 nb_inst_per_class=1,205.4
...,...
PACF: n_random_points=30 nb_inst_per_class=50,35.8
PACF: n_random_points=100 nb_inst_per_class=100,34.8
PACF: n_random_points=50 nb_inst_per_class=100,33.5
ACF&PACF: n_random_points=100 nb_inst_per_class=50,31.8


In [26]:
#generate example cd diagrams
#draw_cd_diagram(pd.read_csv('example_cd_diagram.csv'), labels=True, title="Example CD Diagram", name_file='cd-diagram_example.png')

#### Focus on most accurate lenght method

##### Overall Accuracy

In [27]:
# highest accurate hyperparameters
average_ranks[average_ranks[0]==min_rank][0].index

Index(['ACF&PACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=50'], dtype='object')

In [28]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result[df_result["len_method"]=="ACF&PACF"]
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.940957,0.887078,0.928859,0.950010,0.952057,0.954923
1,0.920222,0.777179,0.907182,0.924210,0.930603,0.916812
10,0.936648,0.878793,0.931548,0.947945,0.947639,0.949764
100,0.946564,0.894728,0.936017,0.949581,0.949840,0.953840
30,0.944603,0.900991,0.932809,0.951538,0.946384,0.951044
50,0.939412,0.901290,0.937588,0.951609,0.949581,0.950849


In [29]:
# create a pivot table with the variatioon of score by hyperparameter
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.001108,0.005683,0.002505,0.001042,0.000958,0.000803
1,0.001477,0.018283,0.003982,0.001853,0.001626,0.003292
10,0.001609,0.005162,0.002392,0.000942,0.000945,0.000767
100,0.001084,0.005170,0.001978,0.000907,0.000850,0.000694
30,0.001021,0.006650,0.002390,0.000914,0.001176,0.000829
50,0.001562,0.002047,0.002035,0.000741,0.000977,0.000793


##### Overall time complexity

In [30]:
# create a pivot table with the mean of time spent in minutes
pd.pivot_table(df_result_acc, values=['time','cweights_time','fsubsequence_time','tdataset_time','tclassifier_time'], index=['dataset_name'], aggfunc='mean')/60

,cweights_time,fsubsequence_time,tclassifier_time,tdataset_time,time
dataset_name,,,,,
Chinatown,0.000148,0.001009,0.000235,0.001189,0.002611
GunPoint,0.000645,0.027494,0.000965,1.169863,1.199090
ItalyPowerDemand,0.000136,0.001336,0.000842,0.009811,0.012150
SmoothSubspace,0.000103,0.002251,0.002864,0.010263,0.015516
SonyAIBORobotSurface2,0.000321,0.002770,0.000283,0.030471,0.033882


In [31]:
# create a pivot table with the mean of time spent in minutes by random points and instances per class
pivot = pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')/60
pivot = pivot.reindex(columns=["(max instances per class)//2","1","10","30","50","100"], index=["(lenthg ts)//2","1","10","30","50","100"])
pivot

nb_per_class,(max instances per class)//2,1,10,30,50,100
rpoint,,,,,,
(lenthg ts)//2,0.293381,0.027368,0.232311,0.570339,0.573378,0.585999
1,0.037193,0.012477,0.013202,0.023076,0.020881,0.021396
10,0.046709,0.011124,0.044932,0.100521,0.103337,0.103388
30,0.126072,0.016894,0.114906,0.277372,0.280085,0.276610
50,0.198972,0.024585,0.184430,0.447486,0.452401,0.453088
100,0.362594,0.032178,0.344944,0.837246,0.842291,0.839293


##### Calculate weights time complexity

In [32]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.015768,0.017721,0.021405,0.013288,0.019234,0.015640
1,0.025683,0.015724,0.017620,0.016451,0.017930,0.018983
10,0.016051,0.017227,0.016891,0.012498,0.013255,0.017242
100,0.016221,0.015707,0.012565,0.015925,0.013062,0.017111
30,0.017597,0.018644,0.014718,0.015531,0.014439,0.015332
50,0.017211,0.013632,0.016124,0.016257,0.010829,0.016805


##### Finding subsequences time complexity

In [33]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.454428,0.048214,0.420566,0.861107,0.844052,0.811984
1,1.042640,0.168519,0.211514,0.502536,0.435071,0.468802
10,0.412867,0.024664,0.211159,0.388084,0.371439,0.451500
100,0.430478,0.026385,0.219369,0.604715,0.510406,0.602022
30,0.416909,0.025748,0.228630,0.537223,0.462311,0.514940
50,0.448713,0.020903,0.233933,0.520430,0.437970,0.544972


##### Transform Dataset time complexity

In [34]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,17.097430,1.372757,13.476520,34.243558,33.315489,33.528118
1,0.865228,0.317757,0.355125,0.752364,0.732387,0.751486
10,2.359592,0.396574,2.452979,5.788271,5.632940,5.718046
100,21.281177,1.751805,20.442802,49.699209,49.671579,49.879559
30,7.116002,0.815828,6.633882,16.026692,16.147576,16.256590
50,11.454217,1.280165,10.797812,26.625319,26.381121,26.558997


##### Classifier time complexity

In [35]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.029010,0.202564,0.016872,0.036527,0.037170,0.042287
1,0.291507,0.245427,0.205218,0.011303,0.196950,0.011906
10,0.012967,0.227461,0.012949,0.012745,0.011951,0.011667
100,0.025279,0.135307,0.019505,0.034469,0.036513,0.035747
30,0.011462,0.151768,0.013443,0.015037,0.015923,0.016329
50,0.016579,0.158979,0.014475,0.020466,0.016005,0.020982
